In [4]:
import sys
!{sys.executable} -m pip install statsmodels --quiet


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import pandas as pd
import statsmodels.api as sm

#load
path = r'C:\Users\Rickey\Downloads\FIN788\DCX RV\spy.csv'
df = pd.read_csv(
    path,
    parse_dates=['Date'],
    dayfirst=True,        #dd
    index_col='Date'
)
df = df.sort_index()     ##ascending
df['rv'] = df['Volatility']

#1,5,22 lags
df['rv_lag1']  = df['rv'].shift(1)
df['rv_week']  = df['rv'].rolling(5).mean().shift(1)
df['rv_month'] = df['rv'].rolling(22).mean().shift(1)
df = df.dropna()

#ols fit
X = sm.add_constant(df[['rv_lag1','rv_week','rv_month']])
y = df['rv']
har_model = sm.OLS(y, X).fit()

#insample forecast
df['har_forecast'] = har_model.predict(X)


start, end = pd.Timestamp('1996-02-02'), pd.Timestamp('2025-04-30')
mask = (df.index >= start) & (df.index <= end)
forecast_df = df.loc[mask, ['har_forecast']]


forecast_df.to_csv('har_forecasts.csv', header=True)

print(f"Saved {len(forecast_df)} rows from {forecast_df.index.min().date()} to {forecast_df.index.max().date()}")


Saved 7279 rows from 1996-02-02 to 2025-04-29
